Transform

In [1]:
import torch
from torchvision import models, transforms
from transformers import BertTokenizer, BertModel
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
from PIL import Image
img=Image.open('testing.png').convert('RGB')
question="what is to the left of bedsheet"
img=transform(img)
img=img.unsqueeze(0).to(device)
inputs = tokenizer(
        question, max_length=512, padding='max_length', truncation=True, return_tensors='pt'
    )
question_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

In [ ]:
from torch.utils.checkpoint import checkpoint
import torch.nn as nn
class VQAModel(nn.Module):
    def __init__(self, num_answers):
        super(VQAModel, self).__init__()
        # Image feature extractor
        self.cnn = models.resnet50(pretrained=True)
        self.cnn.fc = nn.Identity()  # Remove the final classification layer

        # Question feature extractor
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Fusion and final classification
        self.fc1 = nn.Linear(2048 + 768, 1024)
        self.fc2 = nn.Linear(1024, num_answers)
        self.dropout = nn.Dropout(0.5)

    def forward(self, images, input_ids, attention_mask):
        # Extract image features
        image_features = checkpoint(self.cnn, images)  # Use checkpointing for ResNet

        # Extract question features
        outputs = checkpoint(self.bert, input_ids, attention_mask)
        question_features = outputs.last_hidden_state[:, 0, :]  # shape: (batch_size, 768)

        # Concatenate features
        combined_features = torch.cat((image_features, question_features), dim=1)

        # Classification
        x = self.fc1(combined_features)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
model_path="C:/Users/Anish/OneDrive/Desktop/EDI/best_model (1).pth"
model = VQAModel(num_answers=582)
model.to(device)
state_dict = torch.load(model_path)
model.load_state_dict(state_dict["model_state_dict"])
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 147MB/s]


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<ipython-input-7-c7220230b908>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


VQAModel(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0)

In [ ]:
import os
with open("/content/answer_space.txt") as f:
  answer_space = f.read().splitlines()

In [ ]:
output=model(img,question_ids,attention_mask)
# print(output)
_, predicted_idx = torch.max(output, 1)
# Get the top 3 predictions
topk_values, topk_indices = torch.topk(output, k=3, dim=1)

# Convert to readable format (e.g., answer space)
for idx, values in enumerate(topk_indices):
    print(f"Question {idx + 1}:")
    for rank, (val, ans_idx) in enumerate(zip(topk_values[idx], values)):
        print(f"Rank {rank + 1}: {answer_space[ans_idx.item()]} (Score: {val.item():.4f})")

Question 1:
Rank 1: wardrobe (Score: 8.7013)
Rank 2: window (Score: 7.2667)
Rank 3: bed (Score: 6.3726)


In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
!npm install -g localtunnel


added 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!curl ipv4.icanhazip.com

34.82.168.127


In [ ]:
!streamlit run app.py &>./logs.txt &

In [ ]:
!npx localtunnel --port 8501

your url is: https://slick-turtles-rule.loca.lt
^C


#ViT Transformer Testing

In [ ]:
from transformers import ViTForImageClassification, ViTImageProcessor
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Load the pretrained ViT model and image processor
model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name, output_attentions=True)  # Set output_attentions to True
processor = ViTImageProcessor.from_pretrained(model_name)

# Load the image
img_path = "testing.png"
image = Image.open(img_path).convert("RGB")

# Preprocess the image
inputs = processor(images=image, return_tensors="pt")


In [ ]:
model.eval()

# Forward pass to get the attention weights
with torch.no_grad():
    outputs = model(**inputs)

# Get the attention weights (from each layer and each head)
attention = outputs.attentions  # This will be a tuple of attention weights from each layer
print(attention)


(tensor([[[[7.7551e-01, 2.9228e-03, 2.2713e-03,  ..., 1.6110e-03,
           2.2588e-03, 2.4392e-03],
          [3.5704e-01, 4.6516e-01, 1.4657e-03,  ..., 1.6427e-04,
           8.8141e-05, 1.1153e-04],
          [8.4989e-01, 4.8540e-03, 6.2255e-02,  ..., 1.4170e-04,
           1.5141e-04, 1.2412e-04],
          ...,
          [7.0182e-01, 8.0889e-04, 2.8072e-04,  ..., 1.4920e-01,
           9.1595e-03, 9.0073e-03],
          [6.8119e-01, 6.2147e-04, 3.1999e-04,  ..., 5.9967e-03,
           1.5852e-01, 1.8659e-02],
          [6.1656e-01, 3.8810e-04, 2.0847e-04,  ..., 3.0903e-03,
           1.2059e-02, 2.3187e-01]],

         [[8.4897e-01, 7.1466e-04, 9.7898e-04,  ..., 9.7169e-04,
           9.9599e-04, 1.0076e-03],
          [4.9531e-01, 1.1281e-02, 1.4781e-02,  ..., 9.7500e-04,
           1.3131e-03, 8.1040e-04],
          [3.8900e-01, 3.1436e-03, 1.6025e-02,  ..., 1.2373e-04,
           3.9170e-04, 2.4422e-03],
          ...,
          [6.3446e-01, 1.2605e-04, 1.2587e-04,  ..., 6.751

In [ ]:
# Assume the image size is 224x224 and patch size is 16x16
patch_size = 16
image_size = 224

# Calculate the number of patches (for 224x224 image and 16x16 patch)
num_patches = image_size // patch_size

# Reshape the attention map into a grid of patches (num_patches x num_patches)
attention_map = attn_weights.reshape(num_patches, num_patches)

# Normalize the attention map for better visualization
attention_map = (attention_map - attention_map.min()) / (attention_map.max() - attention_map.min())

# Plot the image with the attention map overlay
plt.imshow(image)
plt.imshow(attention_map, cmap='jet', alpha=0.5)  # Heatmap overlay
plt.axis('off')
plt.show()


ValueError: cannot reshape array of size 38809 into shape (14,14)